In [18]:
import torch
from diffusers import StableDiffusionPipeline, DDIMScheduler
from diffusers.utils import load_image

pipeline = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    cache_dir="/vol/paramonos2/projects/antoni/.cache",
).to("cuda")
pipeline.scheduler = DDIMScheduler.from_config(pipeline.scheduler.config)
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter-full-face_sd15.bin")

pipeline.set_ip_adapter_scale(1.0)

image = load_image("/vol/paramonos2/projects/antoni/code/Personal/Old/generating_laugh/bigata_antoni.png")
generator = torch.Generator(device="cpu").manual_seed(26)

image = pipeline(
    prompt="",
    ip_adapter_image=image,
    negative_prompt="",
    num_inference_steps=100,
    generator=generator,
).images[0]
image

  2%|▏         | 2/100 [00:00<00:06, 14.33it/s]

In [4]:
pipeline.vae.__dict__

{'_internal_dict': FrozenDict([('in_channels', 3),
             ('out_channels', 3),
             ('down_block_types',
              ['DownEncoderBlock2D',
               'DownEncoderBlock2D',
               'DownEncoderBlock2D',
               'DownEncoderBlock2D']),
             ('up_block_types',
              ['UpDecoderBlock2D',
               'UpDecoderBlock2D',
               'UpDecoderBlock2D',
               'UpDecoderBlock2D']),
             ('block_out_channels', [128, 256, 512, 512]),
             ('layers_per_block', 2),
             ('act_fn', 'silu'),
             ('latent_channels', 4),
             ('norm_num_groups', 32),
             ('sample_size', 512),
             ('scaling_factor', 0.18215),
             ('force_upcast', True),
             ('_use_default_values', ['force_upcast', 'scaling_factor']),
             ('_class_name', 'AutoencoderKL'),
             ('_diffusers_version', '0.6.0'),
             ('_name_or_path',
              '/vol/paramonos2/projects/